# Les optimisations multipériodes.

# Introduction

Les optimisations multipériodes sont effectuées sur un intervalle de périodes. 
Par exemple, les 12 mois de l'années, ou les 7 jours de la semaine.
L'optimisation la plus connue est l'optimisation de production multipériode linéaire qui permet d'équilibrer les stocks et la production.

Etude globale proposée par <b>Estelle Derrien - Github Estelle15000</b>

# Sommaire

- 1. <b>Optimisation de production multipériode.</b>
        - A/ Description de Notre problème de base donné par Taïwan university
        - Modélisation mathématique
        - Solution avec Python Pulp
           - Sans itérations
           - Explication de la solution
           - Avec itérations
        - B/ Description de Notre problème de base donné par Columbia university
        - Modélisation mathématique
        - Solution avec Python Pulp
            - Avec itérations
            - Explication de la solution
            - Comparaison avec le résultat du solveur Excel
        - C/ Description de Notre problème de base donné par Luc Gibaud
            - Solution avec Pyomo
- 2. <b>Optimisation financière multipériode. ( Cash Flow Matching)</b>
        - Description de Notre problème de base donné par Columbia university
        - Modélisation mathématique
        - Solution avec Python Pulp
            - Avec itérations
            - Explication de la solution
            - traitement du surplus de cash
- 3. <b>Affectation d'employés multipériode. (Multiperiod Work Scheduling)</b>
        - Description de  Notre problème de base
        - Modélisation mathématique
        - Résolution
- LIENS


# 1 Optimisation de production multipériode.

# A/ Notre problème de base, donné par Taïwan university.

HISTOIRE COMPLETE TIREE DU COURS DE TAIWAN UNIVERSITY / COURSERA: 

Le manager prévoit qu'on va vendre 100 , 150, 200 et 170 les jours 1 ,2 , 3 et 4 de la prochaine semaine.

Les coûts de production varient en fonction du jour de la semaine: 
9e 12e 10e ET 12e les jours 1 ,2 , 3 et 4 de la prochaine semaine.

Le coût de stockage est de 1 euro par objet stocké donc, si on le garde 4 jours, ça fait 4 euros de coût. 

QUEL EST LE MEILLEUR PLAN DE PRODUCTION POUR MINIMISER LES COUTS ET NE PAS AVOIR TROP DE STOCKS ?

# Modélisation mathématique

<img src="img/01. Formulation du L.P.png"></img>

# Solution avec Pulp, sans itérations.

In [5]:
# Importer PULP
import pulp

# 1. On veut minimiser Les coûts de production + de stockage.
problem = pulp.LpProblem("Problem",pulp.LpMinimize)


# Les Variables de décision
# production d'objets par jour 1 , 2 , 3 et 4
P1 = pulp.LpVariable('Prodution1', lowBound=0, cat='Integer')
P2 = pulp.LpVariable('Prodution2', lowBound=0, cat='Integer')
P3 = pulp.LpVariable('Prodution3', lowBound=0, cat='Integer')
P4 = pulp.LpVariable('Prodution4', lowBound=0, cat='Integer')


# Stocks par jour 1 , 2 , 3 et 4
S1 = pulp.LpVariable('Stock1', lowBound=0, cat='Integer')
S2 = pulp.LpVariable('Stock2', lowBound=0, cat='Integer')
S3 = pulp.LpVariable('Stock3', lowBound=0, cat='Integer')
S4 = pulp.LpVariable('Stock4', lowBound=0, cat='Integer')

#Objective function : On veut minimiser Le cout de prod + Le cout de stockage
problem += (9*P1 + 12*P2 + 10*P3 + 12*P4) + (S1 + S2 + S3 + S4) 

# Les contraintes de balance d'inventaire - Inventory balancing constraints
problem += P1 - 100 == S1
problem += S1 + P2 - 150 == S2
problem += S2 + P3 - 200 == S3
problem += S3 + P4 - 170 == S4

problem += P1 >= 0
problem += P2 >= 0 
problem += P3 >= 0 
problem += P4 >= 0 


# On résouds 
result = problem.solve()

#On imprime les résultats
print("---Premier jour---")
print("Production:" ,pulp.value(P1))
print("Stock:" ,pulp.value(S1))


print("---2nd jour---")
print("Production:" ,pulp.value(P2))
print("Stock:" ,pulp.value(S2))

print("---3ème jour---")
print("Production3:" ,pulp.value(P3))
print("Stock:" ,pulp.value(S3))

print("---4eme jour---")
print("Production:" ,pulp.value(P4))
print("Stock:" ,pulp.value(S4))

print("Cout global:" ,pulp.value(problem.objective))



---Premier jour---
Production: 250.0
Stock: 150.0
---2nd jour---
Production: 0.0
Stock: 0.0
---3ème jour---
Production3: 370.0
Stock: 170.0
---4eme jour---
Production: 0.0
Stock: 0.0
Cout global: 6270.0


# Explication du résultat : 

Notre intervalle multipériode est ici de 4 jours.

Le solveur calcule ce qu'il faut produire et mettre en stock, afin de se retrouver à la fin de ces 4 jours sans stock sur les bras, tout en satisfaisant la demande.

Le premier jour, on produit 250, ce qui convient aux prévisions du manager qui prévoit de vendre 100 la première journée, et on garde 150 en stock pour le deuxième jour.

Le troisième jour, on produit 370 toujours dans l'optique de convenir aux prévisions du manager, pour les 2 jours suivant, et on garde 170 unités en stock.

Le quatrième jour, on a minimisé notre stock et notre coût de stockage et il ne nous reste plus rien.

# Solution avec Pulp, avec itérations.



In [6]:
# Importer PULP
from pulp import *

# 1. On veut minimiser Les coûts de production, et d'inventaire.
model = LpProblem("Minimiser_le_cout_production",LpMinimize)

# ON définit les couts de production et de stockage pour les jours 1 2 3 ET 4 ; Dans quaters, c'est le nombre de jours SOIT 4. Define production cost, inventory cost, and demande.
# Définir les couts de production par période, le cout de l'inventaire, et la demandee reportée par le service commercial
quaters = list(range(4))
cout_production=[9,12,10,12]
cout_stockage=[1,1,1,1]
demande=[100,150,200,170]


# 2. Définir les variables de décision : Production et Stock - Define Decision Variables: Production and Inventory
x = LpVariable.dicts('quater_prod_', quaters,lowBound=0, cat='Continuous')
y = LpVariable.dicts('quater_stock_', quaters,lowBound=0, cat='Continuous')


# 3. Définir l'objectif , On veut minimiser les coûts de production + de stockage comme dans le problème de université taiwan
model += lpSum([cout_production[i]*x[i] for i in quaters]) + lpSum([cout_stockage[i]*y[i] for i in quaters])

# Définir les contraintes
# Constrainte de capacité de production (Production-capacity constraints)
for i in quaters:
    model.addConstraint(x[i]<=3000)

# Contrainte de balance de stocks ( Inventory-balance constraints)
model.addConstraint(x[0] - y[0] == demande[0]) # (Month 1)

for i in quaters[1:]:
    model.addConstraint(x[i] - y[i] + y[i-1] == demande[i]) # par (jour 2, 3, 4) 

#ON résouds avec le solveur pulp ou un autre entre parenthèses
model.solve()

# On imprime les solutions
for v in model.variables():
    print(v.name, "=", v.varValue)
    
# The optimised objective function value is printed to the screen
print("Notre fonction objectif , le coût global minimisé =  ", value(model.objective))


quater_prod__0 = 250.0
quater_prod__1 = 0.0
quater_prod__2 = 370.0
quater_prod__3 = 0.0
quater_stock__0 = 150.0
quater_stock__1 = 0.0
quater_stock__2 = 170.0
quater_stock__3 = 0.0
Notre fonction objectif , le coût global minimisé =   6270.0


## B/ Notre problème de base, donné par Colombia university.

On le retrouve dans ce lien : 

http://www.columbia.edu/itc/sipa/U6033/client_edit/lectures/lec4.pdf

National Steel Corporation (NSC) produit un acier à usage spécial
utilisé dans les industries aéronautique et aérospatiale. Le service commercial
a reçu des commandes pour les quatre prochains mois :

In [7]:
# Demande  : Janvier  Février     Mars    Avril
# Tonnes:     2300    2000        3100    3000

NSC peut répondre à la demande en produisant de l'acier, en puisant dans ses
inventaire, ou une combinaison de ceux-ci. Inventaire au début de
Janvier est zéro. Les coûts de production devraient augmenter en février et mars.
Les coûts de production et de stocks sont :

In [8]:
#                         Janvier     Février     Mars      Avril
# Coût de production        3000      3300        3600      3600
# Coût d'inventaire         250       250         250       250

Les coûts de production sont en dollars par tonne. 
Les coûts d'inventaire sont en dollars par tonne et par mois. 

Par exemple, 1 tonne en stock pendant 1 mois coûte 250 dollars ; pour 2
mois, cela coûte 500 dollars.

NSC peut produire au maximum 3 000 tonnes d'acier par mois. 
Quel plan de production répond-il à la demande au coût minimum ?

## Modélisation mathématique

A venir !

## Solution avec Pulp

In [9]:
"""
 PLANIFICATION DE PRODUCTION MULTI PERIODE AVEC PRISE EN COMPTE DES STOCKS
Source  :  https://machinelearninggeek.com/solvingmulti-period-production-scheduling-problem-in-python-using-pulp/

La solution EST IDENTIQUE au fichier EXCEL ET A LA DOC COLUMBIA UNIVERSITY :
http://www.columbia.edu/itc/sipa/U6033/client_edit/lectures/lec4.pdf
"""

# Import all classes of PuLP module
from pulp import *

# 1. Initialize Class
model = LpProblem("Minimize Cost",LpMinimize)

# Define production cost, inventory cost, and demand.
quaters = list(range(4))
prod_cost=[3000, 3300, 3600, 3600]
inv_cost=[250, 250, 250, 250]
demand=[2300, 2000, 3100, 3000]

# 2. Define Decision Variables: Production and Inventory
x = LpVariable.dicts('quater_prod_', quaters,lowBound=0, cat='Continuous')
y = LpVariable.dicts('quater_inv_', quaters,lowBound=0, cat='Continuous')

# 3. Define Objective
model += lpSum([prod_cost[i]*x[i] for i in quaters]) + lpSum([inv_cost[i]*y[i] for i in quaters])

# Define Constraints
# Production-capacity constraints
for i in quaters:
    model.addConstraint(x[i]<=3000)

# Inventory-balance constraints
model.addConstraint(x[0] - y[0] == demand[0]) # (Month 1)

for i in quaters[1:]:
    model.addConstraint(x[i] - y[i] + y[i-1] == demand[i]) # for (Month 2, 3, 4) 

    # The problem is solved using PuLP's choice of Solver
model.solve()

# Print the variables optimized value
for v in model.variables():
    print(v.name, "=", v.varValue)
    
# The optimised objective function value is printed to the screen
print("Value of Objective Function = ", value(model.objective))

quater_inv__0 = 700.0
quater_inv__1 = 1700.0
quater_inv__2 = 0.0
quater_inv__3 = 0.0
quater_prod__0 = 3000.0
quater_prod__1 = 3000.0
quater_prod__2 = 1400.0
quater_prod__3 = 3000.0
Value of Objective Function =  35340000.0


c:\Users\secretgirl\AppData\Local\Programs\Python\Python38\lib\site-packages\pulp\pulp.py:1195: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


## C/ Notre problème de base, donné par Luc Gibaud.

Lien :  https://www.quantmetry.com/blog/pyomo-optimisation-python/

Prenons l’exemple d’une entreprise spécialisée dans la vente de T-shirts qui, pour l’année à venir, souhaite établir un planning prévisionnel des T-shirts à produire et à vendre chaque mois, afin d’optimiser ses bénéfices. La société a préalablement fait une estimation de la demande mensuelle de T-shirts de ses clients au cours de l’année.

On sait qu’un T-shirt se vend 10€, coûte 4€ à produire et coûte chaque mois 1,1€ à stocker. La société dispose également d’un stock de 200 T-shirts au début de l’année.

Pour le reste, lire le code sur le lien ainsi que son code Pyomo.

# 2. Optimisation financière multipériode. ( Cash Flow Matching)

## Notre problème de base donné par Colombia University

Une entreprise envisage une rénovation de ses installations sur 3 ans et souhaite
financer le projet en achetant des obligations maintenant (en 2001). 
L'étude de gestion a estimé les besoins de trésorerie suivants pour
le projet:

In [10]:
#                   Année 1    Année 2     Année 3
#                   2002        2003        2004
# besoin en cash    20          30          40
# en millions

Le comité d'investissement envisage quatre obligations d'État pour
achat possible. Le prix et les flux de trésorerie des obligations (en $) sont ( Bond veux dire obligation):

<img src="img/bonds1.jpg">

Quel est le portefeuille d'obligations le moins cher dont les flux de trésorerie sont égaux à
ou dépasser les exigences du projet ?



Note : Différence entre une action et une obligation :
L'action est un titre de capital assorti d'un droit de propriété au sein d'une entreprise. L'obligation, pour sa part, est un titre de créance avec une promesse de remboursement des intérêts. Dans le domaine des actions, on parle surtout d'investissement dont le paiement de dividendes est imprévisible.

In [11]:
#       Bond 1  Bond 2  Bond 3  Bond 4 
# 2001 -1.04    -1.00   -0.98   -0.92 
# 2002  0.05    0.04    1.00    0.00 
# 2003  0.05    1.04            1.00
# 2004  1.05

## Le modèle mathématique .

Lire ce lien : 
https://en.wikipedia.org/wiki/Cashflow_matching

Mais il ne corresponds pas au modèle suivant ... Résolution en cours.

In [13]:
## Solution using Python Pulp

# Import
from pulp import *

# 1. Initialize Class - Xj = # of bond j to purchase today (in millions of bonds)
problem = LpProblem("cashFlow_matching",LpMinimize)

# Define variables
X1 = pulp.LpVariable('X1', lowBound=0, cat='Integer')
X2 = pulp.LpVariable('X2', lowBound=0, cat='Integer')
X3 = pulp.LpVariable('X3', lowBound=0, cat='Integer')
X4 = pulp.LpVariable('X4', lowBound=0, cat='Integer')

# Objective : Minimize the total cost of the bond portfolio (in $ million):
problem += 1.04 * X1 + 1.00  * X2 + 0.98 * X3 + 0.92 * X4 

# In each year, the cash flow from the bonds should equal or exceed 
# the project’s cash requirements: Cash flow from bonds >= Requirement
problem += 0.05  * X1 + 0.04 *  X2 + X3 >= 20,( "year 2002") 
problem +=  0.05 * X1 + 1.04 * X2 + X4 >= 30,( "year 2003") 
problem += 1.05 * X1 >= 40,( "year 2004") 

# The problem is solved using PuLP's choice of Solver
problem.solve()

# Print the variables optimized value
for v in problem.variables():
    print(v.name, "=", v.varValue)
    
# The optimised objective function value is printed to the screen
print("Total cost of the bond portfolio = ", value(problem.objective), " in $ millions")


X1 = 39.0
X2 = 2.0
X3 = 18.0
X4 = 26.0
Total cost of the bond portfolio =  84.12  in $ millions


# 3. Affectation d'employés multipériode. (Multiperiod Work Scheduling)

## Notre problème de base 
Dans le notebook Gestion de projet, nous avons vu que la programmation linéaire pouvait être utilisée pour planifier les employés dans un environnement statique où la demande n’a pas changé au fil du temps. L'exemple suivant (un
version modifiée d'un problème de Wagner [1975]) montre comment LP peut être utilisé pour planifier la formation des employés lorsqu'une entreprise est confrontée à une demande qui évolue dans le temps.

CSL est une chaîne de magasins de services informatiques. Le nombre d'heures de réparation qualifiée qui
Les exigences de CSL au cours des cinq prochains mois sont les suivantes :
- Mois 1 (janvier) : 6 000 heures
- Mois 2 (février) : 7 000 heures
- Mois 3 (mars) : 8 000 heures
- Mois 4 (avril) : 9 500 heures
- Mois 5 (mai) : 11 000 heures

Début janvier, 50 techniciens qualifiés travaillent pour CSL. Chaque technicien qualifié
peut travailler jusqu'à 160 heures par mois. Pour répondre aux demandes futures, les nouveaux techniciens doivent être qualifiés. Il faut un mois pour former un nouveau technicien. Durant le mois de formation, 
le stagiaire doit être encadré pendant 50 heures par un technicien expérimenté. 
Le technicien est payé 2 000 $ par mois (même s'il ne travaille pas les 160 heures complètes).
Pendant le mois de formation, un stagiaire reçoit 1 000 $ par mois. A la fin de chaque mois,
5 % des techniciens expérimentés de CSL ont quitté l’entreprise pour rejoindre Plum Computers. 

Formuler un LP
dont la solution permettra à CSL de minimiser les coûts de main-d'œuvre engagés pour répondre aux exigences de service pour les cinq prochains mois.

# LIENS 

https://www.lindo.com/downloads/LINGO_text/9-Multi-period_Planning_Problems.pdf